# Implementing Shor's Algorithm using the Qrisp framework on Cat Qubits
This tutorial will walk you through the implementation of the famous Shor's algorithm for prime factoring using Qrisp on the Alice&Bob's Qiskit Provider. 


### Why Shor's prime factoring algorithm?
A fair question is to ask why it is of any interest to implement Shor's algorithm and run it with a noise model. It all boils down to the quantum advantage that is provided by the quantum algorithm: the current state of the art classical algorithm for prime factoring is the [general number field sieve](https://en.wikipedia.org/wiki/General_number_field_sieve) (GNFS) which, under some assumptions, achieves the prime factoring of a integer in super-polynomial but sub-exponential time. On the contrary, Shor's algorithm promises a polynomial time to achieve the same task. Equivalently said, Shor's algorithm assures an superpolynomial speedup when compared with a classical algorithm. 
Published in 1994 by Peter Shor, Shor's algorithm shook the field of theoretical computer science, giving a glance of how powerful quantum computation could be. The implications of successfully running the Shor's algorithm on an actual machine are several, but let's just cite the most hype-generating one: the security of the popular RSA cryptosystem is based on the assumption that prime factorization is computationally hard. Well, that's not true anymore with Shor's algorithm.

### Why Cat Qubits?
However, we are far from running the algorithm efficiently on a quantum computer, since qubits and operations between them are noisy; at the moment, the information stored in the system is completaly lost after few operations. People in the field are frenatically trying to solve this problem by designing new and more efficient [quantum error correction codes](https://en.wikipedia.org/wiki/Quantum_error_correction) that are supposed to detect and correct errors. In the quantum realm, the errors on a elementary piece of information are bidimensional: you can have bit-flips ( $|0\rangle \leftrightarrow |1\rangle$ ) and phase-flips ( $|+\rangle \leftrightarrow |-\rangle$ ). Luckily, there exist a technology that is able to take care of one source of error, by exponentially subpressing the bit-flip rate, virtually allowing to only deal with the other type of error: the cat qubits architecture. We have a whole [series of tutorial](https://github.com/Alice-Bob-SW/emulation-examples/blob/main/0%20-%20An%20introduction%20to%20cat%20qubits.ipynb) and [suggested readings]() if you want to know more about it.

Shor's algorithm requires considerably low error rates to give proper results - we will talk about numbers in a following section, when we will do the resource estimation of the algorithm - and therefore error correction is needed. So we tought - wait a second, why don't we use cat qubits then, which are the qubits of the future. And that's what we are going to do.

### Why Qrisp?
Qrisp is a high-level programming language for creating and compiling quantum algorithms which features numerous characteristics to smooth the workflow when writing and executing a quantum algorithm. 
If you want to get familiar with the Qrisp framework, have a look at the dedicated [series of tutorials](https://qrisp.eu/general/tutorial/index.html) available on Qrisp's website.

Qrisp provides full compatibility with Qiskit's framework and it is possible to run Qrisp code directly on the Qiskit's backend making use of the `VirtualQiskitBackend`. This allows us to intantiate the emulator using the Alice & Bob Qiskit Provider as backend and run the code on a cat qubits architechture.




In [1]:
from qrisp import *
from qiskit import *
from qrisp.interface import VirtualQiskitBackend
from qiskit_alice_bob_provider.local.provider import AliceBobLocalProvider

provider = AliceBobLocalProvider()
backend = provider.get_backend('EMU:40Q:LOGICAL_TARGET')
vrtl_qasm_sim = VirtualQiskitBackend(backend)

In [2]:
import math
import random
import numpy as np

We would like to perform the trivial task of prime factoring the number 15. everybody knows that 5x3 = 15, but sometimes is better to double-check - or triple-check, as in this case. 

Why don't we start by comparing a classical brute-force period finding algorithm with its quantum counterpart and compare their performances?
The implementation of the Shor's algorithm is inspired by Qrisp'official tutorial.

## Classical Shor

In [3]:
def is_prime(n):
    # returns True if n is prime
    if n <= 1:
        return False
    elif n <= 3:
        return True
    elif n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def find_period_classical(g,N):
    # returns the period of g in Z_N
    e = 1
    while(pow(g, e, N) != 1):
        e = e+1
    return e

def classical_Shor(N):
    # returns the factors of N by using Shor's algorithm
    # with a classical period finding routine
    if N % 2 == 0:
        return (2, N // 2)
    if (is_prime(N)):
        return (1,N)
    
    while True:
        a = random.randint(2, N - 1)
        if math.gcd(a, N) != 1:
            return math.gcd(a, N), N // math.gcd(a, N)
        
        r = find_period_classical(a, N)
        
        if r % 2 == 0:
            x = pow(a, r // 2, N)
            p = math.gcd(x - 1, N)
            q = math.gcd(x + 1, N)
            if p != 1 and p != N:
                return (p, N // p)
            if q != 1 and q != N:
                return (q, N // q)

In [4]:
# Test the classical Shor's algorithm for N=10000

N=10000
check_list = list()
for i in range(2,N):
    (p,q) = classical_Shor(i)
    check = (p*q == i)
    if(check==False):
        print(i, (p,q))
else:
    print('Test passed!')

Test passed!


## Quantum Shor

In [5]:
from sympy import continued_fraction_convergents, continued_fraction_iterator, Rational

def get_r_candidates(approx):
    rationals = continued_fraction_convergents(continued_fraction_iterator(Rational(approx)))
    return [rat.q for rat in rationals]

In [6]:
def find_order(a, N):
    qg = QuantumModulus(N)
    qg[:] = 1
    qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
    h(qpe_res)
    x=a
    for i in range(len(qpe_res)):
        with control(qpe_res[i]):
            qg *= x
            x = (x*x)%N
    QFT(qpe_res, inv = True)
    return qpe_res.get_measurement(backend=vrtl_qasm_sim)

In [36]:
def find_order_qiskit(a, N):
    qg = QuantumModulus(N)
    qg[:] = 1
    qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
    h(qpe_res)
    x=a
    for i in range(len(qpe_res)):
        with control(qpe_res[i]):
            qg *= x
            x = (x*x)%N
    QFT(qpe_res, inv = True)
    #find size of the registers
    qiskit_qc = qpe_res.qs.compile().to_qiskit()
    cr = ClassicalRegister(qpe_res.size)
    qiskit_qc.add_register(cr)
    qiskit_qc.measure([4,5,6,7,8,9,10,11,12],[0,1,2,3,4,5,6,7,8])
    #qiskit_qc.measure_all()
    results = execute(qiskit_qc, backend=backend, shots=5).result().get_counts()
    return results

In [8]:
def binary_to_float(binaries):
    results = []
    for binary in binaries:
        # Convert binary string to floating-point decimal
        decimal = 0
        for i, bit in enumerate(binary):
            if bit == '1':
                decimal += 2 ** -(i + 1)
        results.append(decimal)
    return results


In [9]:
def quantum_shor_qiskit(a,N):
    meas_res = find_order_qiskit(a,N)
    r_candidates = sum([get_r_candidates(approx) for approx in binary_to_float(meas_res.keys())], [])
    for cand in sorted(set(r_candidates)):
        if (a**cand)%N == 1:
            r = cand
            break
    else:
        raise Exception("Please sample again")
    if r % 2:
        raise Exception("Please choose another a")
    p = np.gcd(a**(r//2)+1, N)
    q = int(N/p)
    if(p>q):
        return(q,p)
    return(p,q)

In [12]:
quantum_shor_qiskit(2,15)

(3, 5)

In [ ]:
# a=2
# N=15
# qg = QuantumModulus(N)
# qg[:] = 1
# qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
# h(qpe_res)
# x=a
# for i in range(len(qpe_res)):
#     with control(qpe_res[i]):
#         qg *= x
#         x = (x*x)%N
# QFT(qpe_res, inv = True)
# #find size of the registers
# qiskit_qc = qpe_res.qs.compile().to_qiskit()
# cr = ClassicalRegister(9)
# qiskit_qc.add_register(cr)
# qiskit_qc.measure([4,5,6,7,8,9,10,11,12],[0,1,2,3,4,5,6,7,8])
# #qiskit_qc.measure_all()
# qc=transpile(qiskit_qc,backend=backend) 
# for _ in range(10):
#     results = execute(qc, backend=backend, shots=10).result().get_counts()
#     r_candidates = sum([get_r_candidates(approx) for approx in binary_to_float(results.keys())], [])
#     for cand in sorted(set(r_candidates)):
#         if (a**cand)%N == 1:
#             r = cand
#             break
#     else:
#         raise Exception("Please sample again")
#     if r % 2:
#         raise Exception("Please choose another a")
#     p = np.gcd(a**(r//2)+1, N)
#     q = int(N/p)
#     print(p,q)


In [75]:
# for _ in range(10):
#     print(quantum_shor_qiskit(2,15))

(1, 15)
(3, 5)
(1, 15)
(3, 5)
(3, 5)
(3, 5)
(1, 15)
(1, 15)
(3, 5)
(1, 15)


Notice that the parameter a is provided manually, instead of it being drawn randomly as in the classical case: this is merely due to optimisation reason, as many runs of the algorithm are needed in order to find the right a. This could take a long time to achieve! 

In [13]:
def quantum_shor(a,N):
    meas_res = find_order(a,N)
    print(meas_res)
    r_candidates = sum([get_r_candidates(approx) for approx in meas_res.keys()], [])
    for cand in sorted(set(r_candidates)):
        if (a**cand)%N == 1:
            r = cand
            break
    else:
        raise Exception("Please sample again")
    if r % 2:
        raise Exception("Please choose another a")
    p = np.gcd(a**(r//2)+1, N)
    q = int(N/p)
    if(p>q):
        return(q,p)
    return(p,q)

In [14]:
N = 15
a = 2
#print(quantum_shor(a,N))


The remarkable aspect of the Qrisp framework is the compactness and simplicity of implementing a fairly complex algorithm, such as Shor's, in a few lines of code as well as the relatively high level of abstraction with respect to gate-level. This task is much more complex to achieve in Qiskit, for example. You can find either a [specific tutorial to factor 15 with an hard-coded circuit](https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/algorithms/shor_algorithm.ipynb); either a [general implementation](https://github.com/Qiskit/qiskit/blob/9c8eb06984c91156eac078f3d2d73b4cf68370b4/qiskit/algorithms/factorizers/shor.py), which has become deprecated. 

In both cases, however, you are forced to operate at gate-level and deal with Qiskit's circuit management technicalities.

### But why is it so slow?
Suppose for the moment to run a noiseless classical simulation of a quantum algorithm. [Gottesman-Knill theorem](https://en.wikipedia.org/wiki/Gottesman%E2%80%93Knill_theorem) states that circuits composed of Clifford gates can be perfectly simulated in polynomial time on a probabilistic classical computer. Shor's algorithm, however, uses non-Clifford gates and as a result the simulation is much longer than in the classical case, in particular for small instances.
On top of that, when adding a noise model, the number of internal operations performed by the Qiskit's backend increases noticeably the computational cost, partially inficiating sparsity assumption of matrix operations and storage.

ERROR CORRECTION?

Anyway, Shor's prime factoring algoritms remains to this day the simplest algorithm to provide a super-polynomial speedup and actually observing it a work could give great insight into the realm of quantum computation.

## Resource Estimation
Let's talk numbers now! How many logical qubits and how many operations would you need to actually solve the problem? Well, for the number of qubits it is quite easy: if $N=2^n$ is the number you want to factorize, you need n qubits to encode its binary representation and $2n +1 $ additional qubits to store the results of the operations. So for $N = 15$, the circuit width of 13. For the number of operations, or depth of the circuit, is a whole different story. In fact, for starters the Qrisp circuit is made of 657 logical gates to implement the quantum sub-routine. Moreover, each of the gate needs to be transpiled into the native universal set of gates of the cat qubits architecture. This step increases the number of gates from the previous 657 to 33567!

We define the quantum volume V as the product of the width and the depth of the circuit. In this case, V = 436371. 

Now, the logical emulator used here adds the noise model to each gate of the circuit. Talk about TARGET and EARLY


Suppose now that the probability that an error occurs is p. Then the probability that no error occurs during the execution of the algorithm is $ (1 - p)^V$. For which error rate the algorithm succeds at least 10% of the times? 
$$
(1-p)^V > 10\%
$$

$$
V ln(1-p) > ln(0.1)
$$

$$
1- p > exp\left(\frac{ln(0.1)}{V}\right)
$$

$$
p < 1 -  exp\left(\frac{ln(0.1)}{V}\right)
$$

##### LOGICAL_TARGET

In [15]:
def error_rate_upper_bound(V, success_rate):
    if success_rate <= 0 or success_rate > 1:
        raise Exception('The success rate should be between 0 and 1')
    return 1 - np.exp(np.log(success_rate)/V)

In [16]:
backend = provider.get_backend('EMU:40Q:LOGICAL_TARGET')
vrtl_qasm_sim = VirtualQiskitBackend(backend)

N = 15
a = 2

In [17]:
qg = QuantumModulus(N)
qg[:] = 1
qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
h(qpe_res)
x=a
for i in range(len(qpe_res)):
    with control(qpe_res[i]):
        qg *= x
        x = (x*x)%N
QFT(qpe_res, inv = True)

<QuantumFloat 'qpe_res'>

In [18]:
qc_qiskit_TARGET = qpe_res.qs.compile().to_qiskit()

In [19]:
print(qc_qiskit_TARGET.width())
print(qc_qiskit_TARGET.depth())

13
651


In [20]:
qc_qiskit_transpiled_TARGET = transpile(qc_qiskit_TARGET,backend=backend) 

In [21]:
print(qc_qiskit_transpiled_TARGET.depth())
print(qc_qiskit_transpiled_TARGET.width())

34607
13


In [22]:
q_volume_TARGET = qc_qiskit_transpiled_TARGET.depth() * (qc_qiskit_transpiled_TARGET.width())  #width * depth of the circuit
print(q_volume_TARGET)

449891


In [23]:
error_rate_upper_bound(q_volume_TARGET, 0.5)

1.5406990728772385e-06

##### LOGICAL_EARLY

In [24]:
backend = provider.get_backend('EMU:15Q:LOGICAL_EARLY')
vrtl_qasm_sim = VirtualQiskitBackend(backend)

N = 15
a = 2



In [29]:
distance = 13
n_qubits = 15 # the maximum number of qubit must be larger than the total number of qubit in order to transpile
kappa_1 = 100
kappa_2= 100000
average_nb_photons=7
t = 1e-3


In [27]:
from qiskit_alice_bob_provider.processor.logical_cat import _cx_error, _ccx_error, _1q_logical_error, _idle_error, _logical_bit_flip_error, _logical_phase_flip_error, _discrete_gate_time

from qiskit_alice_bob_provider.local.backend import ProcessorSimulator
from qiskit_alice_bob_provider.processor.logical_cat import LogicalCatProcessor

In [33]:
print(_1q_logical_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_cx_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_ccx_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_idle_error(t=t, d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_logical_bit_flip_error(d=distance, nbar=average_nb_photons))
print(_logical_phase_flip_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))

{'X': 0.00012951785392367743, 'Y': 1.8380082741908943e-07, 'Z': 0.0014169209214795753}
{'XI': 0.00012931753868677414, 'YI': 1.835165569098732e-07, 'ZI': 0.0014147294795936814, 'IX': 0.00012931753868677414, 'IY': 1.835165569098732e-07, 'IZ': 0.0014147294795936814, 'XX': 1.6774874484995044e-08, 'YX': 2.380548871671667e-11, 'ZX': 1.8351655692959403e-07, 'XY': 2.380548871671667e-11, 'YY': 3.3782744159941896e-14, 'ZY': 2.604312377553646e-10, 'XZ': 1.8351655692959403e-07, 'YZ': 2.604312377553646e-10, 'ZZ': 2.007664897726529e-06}
{'XII': 0.00012911753326193858, 'YII': 1.8323272605984425e-07, 'ZII': 0.0014125414270412828, 'IXI': 0.00012911753326193858, 'IYI': 1.8323272605984428e-07, 'IZI': 0.0014125414270412828, 'IIX': 0.00012911753326193858, 'IIY': 1.8323272605984428e-07, 'IIZ': 0.0014125414270412828, 'IXX': 1.6748930085403117e-08, 'IYX': 2.3768670610429194e-11, 'IZX': 1.8323272607953462e-07, 'IXY': 2.3768670610429194e-11, 'IYY': 3.3730495005137104e-14, 'IZY': 2.6002844892349645e-10, 'IXZ': 1

In [31]:
def logical_error_rate(distance,average_nb_photons,kappa_1,kappa_2):
    return _logical_bit_flip_error(d=distance, nbar=average_nb_photons)+_logical_phase_flip_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2)

In [34]:
print(logical_error_rate(distance=distance,average_nb_photons=average_nb_photons,kappa_1=kappa_1,kappa_2=kappa_2))

0.0015488351907778613


In [35]:
qg = QuantumModulus(N)
qg[:] = 1
qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
h(qpe_res)
x=a
for i in range(len(qpe_res)):
    with control(qpe_res[i]):
        qg *= x
        x = (x*x)%N
QFT(qpe_res, inv = True)

<QuantumFloat 'qpe_res'>

In [37]:
for _ in range(10):
     print(quantum_shor_qiskit(a,N))

(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(1, 15)
(3, 5)


In [27]:
qc_qiskit_EARLY = qpe_res.qs.compile().to_qiskit()

In [28]:
print(qc_qiskit_EARLY.width())
print(qc_qiskit_EARLY.depth())

13
655


In [29]:
qc_qiskit_transpiled_EARLY = transpile(qc_qiskit_EARLY,backend=backend) 

In [30]:
print(qc_qiskit_transpiled_EARLY.depth())
print(qc_qiskit_transpiled_EARLY.width())

33411
13


In [31]:
q_volume_EARLY = qc_qiskit_transpiled_EARLY.depth() * (qc_qiskit_transpiled_EARLY.width())  #width * depth of the circuit
print(q_volume_EARLY)

434343


##### CUSTUM

EMU:15Q:LOGICAL_EARLY

- 1 ≤ nb_shots (int) ≤ 10⁷ . Default 1000
- 4.0 ≤ average_nb_photons (float) ≤ 10⁵. Default 7.0
- 10.0 < kappa_1 (float) ≤ 10⁵. Default 100.0
- 100.0 ≤ kappa_2 (float) ≤ 10⁹. Default 10⁵
- 10⁻⁷ ≤ kappa_1 / kappa_2 ≤ 10⁻¹
- 3 ≤ distance (int) ≤ 300. Default 13. Distance must be an odd number.

EMU:40Q:LOGICAL_TARGET

- 1 ≤ nb_shots (int) ≤ 10⁷ . Default 1000
- 4.0 ≤ average_nb_photons (float) ≤ 10⁵. Default 19.0
- 10.0 < kappa_1 (float) ≤ 10⁵. Default 100.0
- 100.0 ≤ kappa_2 (float) ≤ 10⁹. Default 10⁷
- 10⁻⁷ ≤ kappa_1 / kappa_2 ≤ 10⁻¹
- 3 ≤ distance (int) ≤ 300. Default 15. Distance must be an odd number.

In [ ]:
#backend = provider.get_backend('EMU:15Q:LOGICAL_EARLY', kappa_1=100, kappa_2=1000, distance3)

In [76]:
distance = 3
n_qubits = 15 # the maximum number of qubit must be larger than the total number of qubit in order to transpile
kappa_1 = 100
kappa_2= 1000
average_nb_photons=7
t = 1e-3
name='EMU:15Q:CUSTUM'

backend = ProcessorSimulator(LogicalCatProcessor(distance=distance, n_qubits=n_qubits, kappa_1=kappa_1, kappa_2=kappa_2, average_nb_photons = 7), translation_stage_plugin='sk_synthesis')
backend.name = name
vrtl_qasm_sim = VirtualQiskitBackend(backend)

print(_1q_logical_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_cx_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_ccx_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_idle_error(t=t, d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))
print(_logical_bit_flip_error(d=distance, nbar=average_nb_photons))
print(_logical_phase_flip_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2))

{'X': 2.494573711431247e-06, 'Y': 2.494573711431247e-06, 'Z': 0.49999750542628857}
{'XI': 1.247280632817622e-06, 'YI': 1.247280632817622e-06, 'ZI': 0.2499975054325115, 'IX': 1.247280632817622e-06, 'IY': 1.247280632817622e-06, 'IZ': 0.2499975054325115, 'XX': 6.222898001763867e-12, 'YX': 6.222898001763867e-12, 'ZX': 1.2472806328176218e-06, 'XY': 6.222898001763867e-12, 'YY': 6.222898001763867e-12, 'ZY': 1.2472806328176218e-06, 'XZ': 1.2472806328176218e-06, 'YZ': 1.2472806328176218e-06, 'ZZ': 0.24999750543251148}
{'XII': 6.236372049753334e-07, 'YII': 6.236372049753334e-07, 'ZII': 0.12499812907905074, 'IXI': 6.236372049753337e-07, 'IYI': 6.236372049753337e-07, 'IZI': 0.1249981290790508, 'IIX': 6.236372049753337e-07, 'IIY': 6.236372049753337e-07, 'IIZ': 0.1249981290790508, 'IXX': 3.11143347740417e-12, 'IYX': 3.11143347740417e-12, 'IZX': 6.236372049753336e-07, 'IXY': 3.11143347740417e-12, 'IYY': 3.11143347740417e-12, 'IZY': 6.236372049753336e-07, 'IXZ': 6.236372049753336e-07, 'IYZ': 6.2363720

In [77]:
qg = QuantumModulus(N)
qg[:] = 1
qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
h(qpe_res)
x=a
for i in range(len(qpe_res)):
    with control(qpe_res[i]):
        qg *= x
        x = (x*x)%N
QFT(qpe_res, inv = True)

<QuantumFloat 'qpe_res'>

In [78]:
qc_circuit = qpe_res.qs.compile().to_qiskit()

In [79]:
qc_qiskit_transpiled_CUSTUM= transpile(qc_circuit,backend=backend)

In [81]:
for _ in range(4):
    print(quantum_shor_qiskit(a,N))

(3, 5)
(1, 15)
(3, 5)
(3, 5)


In [ ]:
# shots = 2
# transpile once


## Further readings:
- Shor's paper
- Bitcoin paper
- Elliptic curve paper
